In [1]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import glob

import pyomo
import pyomo.opt
import pyomo.environ as pe

from src.parsers import HMParser, CotevParser
from src.resources import Aggregator, Generator, Load, Storage, Vehicle

In [2]:
# Data parsing

# EC data for non-renewable generators and batteries
data_ec = HMParser(file_path='/Users/ecgomes/DataspellProjects/pyecom/data/EC_V4.xlsx', ec_id=1)
data_ec.parse()

# EV data from the EV4EU simulator
data_ev = CotevParser(population_path=
                      '/Users/ecgomes/DataspellProjects/pyecom/data/simulation_dataframes_2years/population_731.csv',
                      driving_history_path=
                      '/Users/ecgomes/DataspellProjects/pyecom/data/simulation_dataframes_2years/ev_driving_history_731.csv',
                      assigned_segments_path='/Users/ecgomes/DataspellProjects/pyecom/data/simulation_dataframes_2years/assigned_segments_731.csv',
                      parse_date_start='2019',
                      parse_date_end='2020')
data_ev.parse()

In [3]:
# UPAC Data load

data_upacs = {}
for i in glob.glob('/Users/ecgomes/Documents/PhD/UPAC data/upac*_pv.csv'):
    temp = pd.read_csv(i, index_col=0, parse_dates=True)
    temp = temp.resample('H').mean()

    # Need to divide by 1000 to convert from W to kW
    temp['pv'] = temp['pv'] / 1000
    temp['load'] = temp['load'] / 1000

    # We only want 2019 and 2020 data
    temp = temp.loc['2019':'2020']

    # Fill potential NaN values with interpolation
    temp = temp.interpolate()

    name = i.split('/')[-1].split('_')[0].split('upac')[1]

    data_upacs[name] = temp

In [4]:
# Train resource creation

def create_resources(upacs, ec, ev):
    """
    Create the resources for the training environment.
    return a list of resources.
    :param upacs: dict with the UPAC data
    :param ec: dict with the EC data
    :param ev: dict with the EV data
    """

    resources = []

    # Add generators (from pv column from the UPAC data)
    for i in range(len(upacs)):
        current_name = list(upacs.keys())[i]
        resources.append(Generator(
            name='generator_' + current_name,
            value=np.zeros(upacs[current_name]['pv'].shape),
            lower_bound=np.zeros(upacs[current_name]['pv'].shape),
            upper_bound=upacs[current_name]['pv'].values * 5, # 5 times the PV generation
            cost=ec.generator['cost_parameter_b'][0] * np.ones(upacs[current_name].shape[0]),
            cost_nde=ec.generator['cost_nde'][0],
            is_renewable=True))

    # Add loads (from load column from the UPAC data)
    for i in range(len(upacs)):
        current_name = list(upacs.keys())[i]
        resources.append(Load(
            name='load_' + current_name,
            value=upacs[current_name]['load'],
            lower_bound=np.zeros(upacs[current_name].shape),
            upper_bound=upacs[current_name]['load'].values,
            cost=np.ones(upacs[current_name].shape[0]),
            cost_cut=ec.load['cost_cut'][0],
            cost_reduce=ec.load['cost_reduce'][0],
            cost_ens=ec.load['cost_ens'][0]))

    # Add storage (from the EC data)
    for i in range(ec.storage['p_charge_limit'].shape[0]):
        resources.append(Storage(
            name='storage_{:02d}'.format(i+1),
            value=ec.storage['initial_state'][i] * np.ones(upacs['02'].shape[0]),
            lower_bound=np.ones(upacs['02'].shape[0]) * ec.storage['energy_min_percentage'][i],
            upper_bound=(ec.storage['energy_capacity'][i] * np.ones(upacs['02'].shape[0])),
            cost=np.ones(upacs['02'].shape[0]),
            cost_discharge=np.tile(ec.storage['discharge_price'][i], (int(upacs['02'].shape[0] / 24))),
            cost_charge=np.tile(ec.storage['charge_price'][i], (int(upacs['02'].shape[0] / 24))),
            capacity_max=ec.storage['energy_capacity'][i],
            capacity_min=ec.storage['energy_min_percentage'][i],
            initial_charge=ec.storage['initial_state'][i],
            discharge_efficiency=ec.storage['discharge_efficiency'][i],
            discharge_max=np.tile(ec.storage['p_discharge_limit'][i], (int(upacs['02'].shape[0] / 24))),
            charge_efficiency=ec.storage['charge_efficiency'][i],
            charge_max=np.tile(ec.storage['p_charge_limit'][i], (int(upacs['02'].shape[0] / 24))),
            capital_cost=np.array([0.05250, 0.10500, 0.01575])))

    # Add vehicles (from the EV data)
    for i in np.arange(len(ev)):
        # Append to the list of resources
        resources.append(ev[i])

    # Append Aggregator
    resources.append(Aggregator(
        name='aggregator',
        value=np.zeros(upacs['02'].shape[0]),
        lower_bound=np.zeros(upacs['02'].shape[0]),
        upper_bound=np.tile(ec.peers['import_contracted_p_max'][0, 0], (upacs['02'].shape[0])),
        cost=np.tile(ec.peers['buy_price'][0, 0], (upacs['02'].shape[0])),
        imports=np.zeros(upacs['02'].shape[0]),
        exports=np.zeros(upacs['02'].shape[0]),
        import_cost=np.tile(ec.peers['buy_price'][0, 0], (int(upacs['02'].shape[0]))), # * 100,
        export_cost=np.tile(ec.peers['sell_price'][0, 0], (int(upacs['02'].shape[0]))),
        import_max=np.tile(ec.peers['import_contracted_p_max'][0, 0], (int(upacs['02'].shape[0]))),
        export_max=np.tile(ec.peers['export_contracted_p_max'][0, 0], (int(upacs['02'].shape[0])))))

    return resources

In [5]:
# Create resources for the training environment

def iterate_resources(u, c, e):

    temp = {}

    # Save first key of upac data
    first_key = list(u.keys())[0]

    # Loop to iterate over days in the datasets
    for i in np.unique(u[first_key].index.date):
        # Create the resources for the training environment

        date = i.strftime('%Y-%m-%d')

        temp_u = {k: v.loc[date] for k, v in u.items()}
        temp_e = e.create_resources(e.population, e.trips_grid, e.assigned_segments, date)

        temp[date] = create_resources(upacs=temp_u,
                                      ec=c,
                                      ev=temp_e)

    return temp

dataset_resources = iterate_resources(u=data_upacs, c=data_ec, e=data_ev)

In [6]:
# Auxiliary function to convert numpy arrays to dictionaries

def convert_to_dictionary(a):
    temp_dictionary = {}

    if len(a.shape) == 3:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                for dim2 in np.arange(a.shape[2]):
                    temp_dictionary[(dim0+1, dim1+1, dim2+1)] = a[dim0, dim1, dim2]
    elif len(a.shape) == 2:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                temp_dictionary[(dim0+1, dim1+1)] = a[dim0, dim1]

    else:
        for dim0 in np.arange(a.shape[0]):
            temp_dictionary[(dim0+1)] = a[dim0]

    return temp_dictionary

In [11]:
# Create the Pyomo model

class HMMilpPyomo:
    
    """
    Class to create the Pyomo model for the HM MILP problem.
    Considers Generators, Loads, Storage, Vehicles and Imports/Exports.
    """
    
    def __init__(self, resources, default_behaviour=pe.Constraint.Skip):
        
        self.resources = resources
        self.default_behaviour = default_behaviour
        
        # Resource types
        self.generators = self.separate_resources(Generator)
        self.loads = self.separate_resources(Load)
        self.storages = self.separate_resources(Storage)
        self.vehicles = self.separate_resources(Vehicle)
        self.aggregator = self.separate_resources(Aggregator)[0]
        
        # Model variables
        self.model = None
        
    def set_resources(self, resources):
        """
        Set the resources for the Pyomo model. 
        To be used if resources need to be updated after class is instantiated.
        :param resources: list of resources
        """
        self.resources = resources
        return

    def separate_resources(self, type):
        """
        Separate resources by type.
        :param type: type of resource to separate
        """
        temp = [res for res in self.resources if isinstance(res, type)]

        return temp
        
    def create_model(self):
        """
        Create the Pyomo model.
        """
        # Create the model
        self.model = pe.ConcreteModel()
        return
    
    def solve_model(self, 
                    write_path: str = None, 
                    solver_path: str = None,
                    solver_io_options=None):
        
        if solver_io_options is None:
            solver_io_options = {'symbolic_solver_labels': True}
            
        # Create the model file
        self.model.write(write_path + '_model.lp', io_options=solver_io_options)
        
        # Create the solver
        solver = pe.SolverFactory('scip', executable=solver_path)
        solver.options['LogFile'] = write_path + '_log.log'
        
        # Solve the model
        results = solver.solve(self.model)
        results.write()
        
        return

    @staticmethod
    def extract_results(vals):
        # make a pd.Series from each
        s = pd.Series(vals.extract_values(),
                      index=vals.extract_values().keys())
    
        # if the series is multi-indexed we need to unstack it...
        if type(s.index[0]) == tuple:    # it is multi-indexed
            s = s.unstack(level=1)
        else:
            # force transition from Series -> df
            s = pd.DataFrame(s)
    
        return s
    
    def add_model_sets(self):
        """
        Add the sets to the Pyomo model (time dimension and resource numbers).
        """
        self.model.t = pe.Set(initialize=np.arange(1, self.generators[0].upper_bound.shape[0] + 1))
        self.model.gen = pe.Set(initialize=np.arange(1, len(self.generators) + 1))
        self.model.loads = pe.Set(initialize=np.arange(1, len(self.loads) + 1))
        self.model.stor = pe.Set(initialize=np.arange(1, len(self.storages) + 1))
        self.model.v2g = pe.Set(initialize=np.arange(1, len(self.vehicles) + 1))
        
        return
    
    def add_model_aggregator(self):
        # Parameters (information to pass to the model)
        self.model.impMax = pe.Param(self.model.t,
                                     initialize=convert_to_dictionary(self.aggregator.import_max),
                                     doc='Maximum import power')
        self.model.expMax = pe.Param(self.model.t,
                                     initialize=convert_to_dictionary(self.aggregator.export_max),
                                     doc='Maximum export power')
        self.model.impCOst = pe.Param(self.model.t,
                                       initialize=convert_to_dictionary(self.aggregator.import_cost),
                                       doc='Buy price')
        self.model.sellPrice = pe.Param(self.model.t,
                                        initialize=convert_to_dictionary(self.aggregator.export_cost),
                                        doc='Sell price')
        
        # Variables (values to be optimized)
        self.model.imports = pe.Var(self.model.t, within=pe.NonNegativeReals, initialize=0,
                                    doc='Import power')
        self.model.exports = pe.Var(self.model.t, within=pe.NonNegativeReals, initialize=0,
                                    doc='Export power')
        self.model.import_binary = pe.Var(self.model.t, within=pe.Binary, initialize=0,
                                         doc='Binary variable for import')
        self.model.export_binary = pe.Var(self.model.t, within=pe.Binary, initialize=0,
                                         doc='Binary variable for export')
        
        # Constraints
        def impMax_rule(model, t):
            return model.imports[t] <= model.impMax[t] * model.import_binary[t]
        self.model.impMax_rule = pe.Constraint(self.model.t, rule=impMax_rule)
        
        def expMax_rule(model, t):
            return model.exports[t] <= model.expMax[t] * (1 - model.export_binary[t])
        self.model.expMax_rule = pe.Constraint(self.model.t, rule=expMax_rule)
        
        def impExp_rule(model, t):
            return model.import_binary[t] + model.export_binary[t] <= 1
        self.model.impExp_rule = pe.Constraint(self.model.t, rule=impExp_rule)
        
        return
    
    def add_model_generators(self):
        # Parameters (information to pass to the model)
        self.model.genMin = pe.Param(self.model.gen, self.model.t,
                                     initialize=convert_to_dictionary(np.concatenate([res.lower_bound.reshape(1, -1) for res in self.generators], axis=0)),
                                     doc='Minimum generation power')
        self.model.genMax = pe.Param(self.model.gen, self.model.gen,
                                     initialize=convert_to_dictionary(np.concatenate([res.upper_bound.reshape(1, -1) for res in self.generators], axis=0)),
                                     doc='Maximum generation power')
        self.model.genCost = pe.Param(self.model.gen, self.model.t,
                                      initialize=convert_to_dictionary(np.concatenate([res.cost.reshape(1, -1) for res in self.generators], axis=0)),
                                      doc='Generation cost')
        self.model.genCostNDE = pe.Param(self.model.gen,
                                         initialize=convert_to_dictionary(np.array([res.cost_nde.reshape(1, -1) for res in self.generators])),
                                         doc='Generation cost NDE')
        self.model.genType = pe.Param(self.model.gen,
                                      initialize=convert_to_dictionary(np.array([res.is_renewable for res in self.generators])),
                                      doc='Generation type')
        
        # Variables (values to be optimized)
        self.model.genPower = pe.Var(self.model.gen, self.model.t, within=pe.NonNegativeReals, initialize=0,
                                     doc='Generation power')
        self.model.genExcess = pe.Var(self.model.gen, self.model.t, within=pe.NonNegativeReals, initialize=0,
                                      doc='Excess generation power')
        self.model.gen_binary = pe.Var(self.model.gen, self.model.t, within=pe.Binary, initialize=0,
                                       doc='Binary variable for generation')
        
        # Constraints
        def genMin_rule(model, g, t):
            if model.genType[g]:
                return model.genPower[g, t] >= model.genMin[g, t] * model.gen_binary[g, t]
            return pe.Constraint.Skip
        self.model.genMin_rule = pe.Constraint(self.model.gen, self.model.t, rule=genMin_rule)
        
        def genMax_rule(model, g, t):
            if model.genType[g]:
                return model.genPower[g, t] <= model.genMax[g, t] * model.gen_binary[g, t]
            else:
                return model.genPower[g, t] + model.genExcess[g, t] == model.genMax[g, t]
        self.model.genMax_rule = pe.Constraint(self.model.gen, self.model.t, rule=genMax_rule)
        
        return
        
    def add_model_loads(self):
        # Parameters (information to pass to the model)
        self.model.loadMax = pe.Param(self.model.loads, self.model.t,
                                      initialize=convert_to_dictionary(np.concatenate([res.upper_bound.reshape(1, -1) for res in self.loads], axis=0)),
                                      doc='Maximum load power')
        self.model.loadCut = pe.Param(self.model.loads, self.model.t,
                                      initialize=convert_to_dictionary(np.concatenate([res.upper_bound.reshape(1, -1) for res in self.loads], axis=0)),
                                      doc='Load cut power')
        self.model.loadReduce = pe.Param(self.model.loads, self.model.t,
                                         initialize=convert_to_dictionary(np.concatenate([res.upper_bound.reshape(1, -1) for res in self.loads], axis=0)),
                                         doc='Load reduce power')
        self.model.loadEns = pe.Param(self.model.loads, self.model.t,
                                      initialize=convert_to_dictionary(np.concatenate([res.upper_bound.reshape(1, -1) for res in self.loads], axis=0)),
                                      doc='Load ens power')
        self.model.loadCutCost = pe.Param(self.model.loads,
                                     initialize=convert_to_dictionary(np.array([res.cost_cut for res in self.loads])),
                                     doc='Load cut cost')
        self.model.loadReduceCost = pe.Param(self.model.loads,
                                             initialize=convert_to_dictionary(np.array([res.cost_reduce for res in self.loads])),
                                             doc='Load reduce cost')
        self.model.loadEnsCost = pe.Param(self.model.loads,
                                          initialize=convert_to_dictionary(np.array([res.cost_ens for res in self.loads])),
                                          doc='Load ENS cost')
        
        # Variables (values to be optimized)
        self.model.loadCutPower = pe.Var(self.model.loads, self.model.t, within=pe.NonNegativeReals, initialize=0,
                                   doc='Load cut power')
        self.model.loadReducePower = pe.Var(self.model.loads, self.model.t, within=pe.NonNegativeReals, initialize=0,
                                      doc='Load reduce power')
        self.model.loadEnsPower = pe.Var(self.model.loads, self.model.t, within=pe.NonNegativeReals, initialize=0,
                                   doc='Load ENS power')
        self.model.load_binary = pe.Var(self.model.loads, self.model.t, within=pe.Binary, initialize=0,
                                       doc='Binary variable for load')
        
        # Constraints    
        def loadReduce_rule(model, l, t):
            return model.loadReducePower[l, t] <= model.loadReduce[l, t]
        self.model.loadReduce_rule = pe.Constraint(self.model.loads, self.model.t, rule=loadReduce_rule)
        
        def loadCut_rule(model, l, t):
            return model.loadCutPower[l, t] <= model.loadCut[l, t] * model.load_binary[l, t]
        self.model.loadCut_rule = pe.Constraint(self.model.loads, self.model.t, rule=loadCut_rule)
        
        def loadEns_rule(model, l, t):
            return model.loadEnsPower[l, t] + model.loadCutPower[l, t] + model.loadReducePower[l, t] <= model.loadMax[l, t]
        self.model.loadEns_rule = pe.Constraint(self.model.loads, self.model.t, rule=loadEns_rule)
        
        return
    
    def add_model_storages(self):
        # Parameters (information to pass to the model)
        self.model.storageDischargeMax = pe.Param(self.model.stor, self.model.t,
                                                  initialize=convert_to_dictionary(np.concatenate([res.discharge_max.reshape(1, -1) for res in self.storages], axis=0)),
                                                  doc='Maximum discharge power')
        self.model.storageChargeMax = pe.Param(self.model.stor, self.model.t,
                                               initialize=convert_to_dictionary(np.concatenate([res.charge_max.reshape(1, -1) for res in self.storages], axis=0)),
                                               doc='Maximum charge power')
        self.model.storageCapacityMax = pe.Param(self.model.stor,
                                                 initialize=convert_to_dictionary(np.array([[res.capacity_max] for res in self.storages])),
                                                 doc='Maximum storage capacity')
        self.model.storageCapacityMin = pe.Param(self.model.stor,
                                                 initialize=convert_to_dictionary(np.concatenate([res.lower_bound.reshape(1, -1) for res in self.storages], axis=0)),
                                                 doc='Minimum storage capacity')
        self.model.storageInitialCharge = pe.Param(self.model.stor,
                                                   initialize=convert_to_dictionary(np.array([[res.initial_charge] for res in self.storages])),
                                                   doc='Initial storage charge')
        self.model.storageDischargeEfficiency = pe.Param(self.model.stor,
                                                        initialize=convert_to_dictionary(np.array([[res.discharge_efficiency] for res in self.storages])),
                                                        doc='Discharge efficiency')
        self.model.storageChargeEfficiency = pe.Param(self.model.stor,
                                                      initialize=convert_to_dictionary(np.array([[res.charge_efficiency] for res in self.storages])),
                                                      doc='Charge efficiency')
        self.model.storageDischargeCost = pe.Param(self.model.stor, self.model.t,
                                                   initialize=convert_to_dictionary(np.concatenate([res.cost_discharge.reshape(1, -1) for res in self.storages], axis=0)),
                                                   doc='Discharge cost')
        self.model.storageChargeCost = pe.Param(self.model.stor, self.model.t,
                                                initialize=convert_to_dictionary(np.concatenate([res.cost_charge.reshape(1, -1) for res in self.storages], axis=0)),
                                                doc='Charge cost')
        
        # Variables (values to be optimized)
        self.model.storageState = pe.Var(self.model.stor, self.model.t, within=pe.NonNegativeReals, initialize=0,
                                         doc='Storage state of charge')
        self.model.storageDischargePower = pe.Var(self.model.stor, self.model.t, within=pe.NonNegativeReals, initialize=0,
                                                  doc='Storage discharge power')
        self.model.storageChargePower = pe.Var(self.model.stor, self.model.t, within=pe.NonNegativeReals, initialize=0,
                                               doc='Storage charge power')
        self.model.storageRelaxPower = pe.Var(self.model.stor, self.model.t, within=pe.NonNegativeReals, initialize=0,
                                              doc='Storage relaxation power')
        self.model.storageDischarge_binary = pe.Var(self.model.stor, self.model.t, within=pe.Binary, initialize=0,
                                                    doc='Binary variable for storage discharge')
        self.model.storageCharge_binary = pe.Var(self.model.stor, self.model.t, within=pe.Binary, initialize=0,
                                                 doc='Binary variable for storage charge')
        
        # Constraints
        def storageDischarge_rule(model, s, t):
            return model.storageDischargePower[s, t] <= model.storageDischargeMax[s, t] * model.storageDischarge_binary[s, t]
        self.model.storageDischarge_rule = pe.Constraint(self.model.stor, self.model.t, rule=storageDischarge_rule)
        
        def storageCharge_rule(model, s, t):
            return model.storageChargePower[s, t] <= model.storageChargeMax[s, t] * model.storageCharge_binary[s, t]
        self.model.storageCharge_rule = pe.Constraint(self.model.stor, self.model.t, rule=storageCharge_rule)
        
        def storageState_rule(model, s, t):
            return model.storageState[s, t] <= model.storageCapacityMax[s]
        self.model.storageState_rule = pe.Constraint(self.model.stor, self.model.t, rule=storageState_rule)
        
        def storageStateMin_rule(model, s, t):
            return model.storageState[s, t] >= model.storageCapacityMin[s] * model.storageCapacityMax[s] - model.storageRelaxPower[s, t]
        self.model.storageStateMin_rule = pe.Constraint(self.model.stor, self.model.t, rule=storageStateMin_rule)
        
        def storageBalance_rule(model, s, t):
            if t == 1:
                return model.storageState[s, t] == model.storageCapacityMax[s] * model.storageInitialCharge[s] \
                    + model.storageChargePower[s, t] * model.storageChargeEfficiency[s] \
                    - model.storageDischargePower[s, t] / model.storageDischargeEfficiency[s]
            elif t > 1:
                return model.storageState[s, t] == model.storageState[s, t-1] \
                    + model.storageChargePower[s, t] * model.storageChargeEfficiency[s] \
                    - model.storageDischargePower[s, t] / model.storageDischargeEfficiency[s]
        self.model.storageBalance_rule = pe.Constraint(self.model.stor, self.model.t, rule=storageBalance_rule)
        
        def storageChargeDischarge_rule(model, s, t):
            return model.storageCharge_binary[s, t] + model.storageDischarge_binary[s, t] <= 1
        self.model.storageChargeDischarge_rule = pe.Constraint(self.model.stor, self.model.t, rule=storageChargeDischarge_rule)
            
        return
    

In [41]:
dataset_resources['2019-01-01'][11].__dict__

{'name': 'storage_02',
 'value': array([0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8,
        0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8]),
 'lower_bound': array([0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2,
        0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]),
 'upper_bound': array([100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
        100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
        100., 100.]),
 'cost': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1.]),
 'capacity_max': 100.0,
 'capacity_min': 0.2,
 'initial_charge': 0.8,
 'charge_efficiency': 0.96,
 'discharge_efficiency': 0.96,
 'capital_cost': array([0.0525 , 0.105  , 0.01575]),
 'discharge': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.]),
 'discharge_max': array([20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 